## Reading data

In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
#PRJ = "/home/weiyi/workspace/iceburger"
PRJ = "/workspace/lettuce"
DATA = os.path.join(PRJ, "data/processed")

In [3]:
def image_normalization(x, percentile=1):
    """Normalize the image signal value by rescale data
    
    :param x: :class:`numpy.ndarray` of signal of dimension (height, width, 2)
    :param percentile: signal greater or less than the percentile will be capped
        as 1 and 0 respectively
    :returns: :class:`numpy.ndarray` of normalized 3 channel image with last
        channel totally black
    """
    vmax = np.percentile(x, 100 - percentile)
    vmin = np.percentile(x, percentile)
    x = (x - vmin) / (vmax - vmin)
    x[x > 1] = 1
    x[x < 0] = 0
    return np.concatenate([x, np.zeros(x.shape[:2] + (1,))], axis=-1)[np.newaxis, :, :, :]

In [4]:
def parse_json_data(json_filename):
    """Parse json data to generate trainable matrices
    
    :param json_filename: path to input json file
    :returns: a `tuple` of
        X: :class:`numpy.ndarray` of dimension (nb_samples, height, width, 3)
        X_angle: :class:`numpy.array` of dimension (nb_samples) of incidence
            angles
        y: :class:`numpy.array` of labels
    """
    df = pd.read_json(json_filename)
    dim = int(np.sqrt(len(df.band_1.iloc[0])))
    _X = np.concatenate([
        np.concatenate([np.array(r.band_1).reshape((dim, dim, 1)),
                        np.array(r.band_2).reshape((dim, dim, 1))],
                       axis=-1)[np.newaxis, :, :, :]
        for _, r in df.iterrows()], axis = 0)
    X = np.concatenate([image_normalization(x) for x in _X], axis=0)
    X_angle = df.inc_angle.values
    y = df.is_iceberg.values
    return (X, X_angle, y)

In [5]:
X_train, X_train_angle, y = parse_json_data(os.path.join(DATA, "train.json"))

In [24]:
print(X_train.shape)
print(X_train_angle.shape)
print(y.shape)
json=os.path.join(DATA, "train.json")
df_json = pd.read_json(json)
#df_json.head()
len(df_json.band_1.iloc[0])
df_json.head()
dim = int(np.sqrt(len(df_json.band_1.iloc[0])))
print(dim)


(1604, 75, 75, 3)
(1604,)
(1604,)
75


In [6]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [7]:
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 45)

In [8]:
genX1 = gen.flow(X_train, y,  batch_size=32, seed=666)
#genX2 = gen.flow(X_train_angle, y, batch_size=32, seed = 666)

In [14]:
genX1.next()

(array([[[[ 0.57730412,  0.20129545,  0.        ],
          [ 0.57730412,  0.20129545,  0.        ],
          [ 0.57730412,  0.20129545,  0.        ],
          ..., 
          [ 0.49286973,  0.13015991,  0.        ],
          [ 0.49286973,  0.13015991,  0.        ],
          [ 0.52832937,  0.36815003,  0.        ]],
 
         [[ 0.59257483,  0.20129545,  0.        ],
          [ 0.59257483,  0.20129545,  0.        ],
          [ 0.59257483,  0.20129545,  0.        ],
          ..., 
          [ 0.49286973,  0.13015991,  0.        ],
          [ 0.52832937,  0.36815003,  0.        ],
          [ 0.52832937,  0.36815003,  0.        ]],
 
         [[ 0.70029521,  0.1670571 ,  0.        ],
          [ 0.70029521,  0.1670571 ,  0.        ],
          [ 0.70029521,  0.1670571 ,  0.        ],
          ..., 
          [ 0.49286973,  0.13015991,  0.        ],
          [ 0.52832937,  0.36815003,  0.        ],
          [ 0.39123344,  0.34393492,  0.        ]],
 
         ..., 
         [

In [26]:
def exponential(args):
    X,angles,Y = args
    return (X[0]+j*X[1])*np.exp(-j*pi*angles/180)

In [27]:
X,X1,Y = genX1.next()


ValueError: not enough values to unpack (expected 3, got 2)